In [1]:
import pandas as pd
import numpy as np
import re

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Load dataframe
df = pd.read_pickle('../data/01_df_clean_2018.pkl')

df.head()

,case_id,target,opened,closed,status,status_notes,responsible_agency,category,request_type,request_details,address,street,supervisor_district,neighborhood,police_district,latitude,longitude,point,source,has_media
810969,9995194,1,2018-12-31 10:42:22,2018-12-31 00:00:00,Closed,Case Resolved - duplicate of 21375555 -,DPW - Bureau of Street Use and Mapping - G,General Request - PUBLIC WORKS,request_for_service,bsm - request_for_service,"31 HANGAH ST, SAN FRANCISCO, CA, 94108",HANGAH ST,3.0,Chinatown,CENTRAL,37.793526,-122.407562,"(37.7935257, -122.40756226)",Integrated Agency,0
811474,9993791,1,2018-12-30 22:40:00,2018-12-30 23:13:06,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Black - Toyota Camry - NO PLATES,"140 PRECITA AVE, SAN FRANCISCO, CA, 94110",PRECITA AVE,9.0,Bernal Heights,INGLESIDE,37.747047,-122.417297,"(37.747047, -122.417297)",Mobile/Open311,1
811481,9993771,1,2018-12-30 22:18:00,2018-12-30 22:19:29,Closed,Case is Invalid - Contact name and phone numbe...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,silver` - Mazda 3 Hatch back - 7YVN660,"754 MOULTRIE ST, SAN FRANCISCO, CA, 94110",MOULTRIE ST,9.0,Bernal Heights,INGLESIDE,37.735334,-122.416215,"(37.73533399, -122.4162146)",Phone,0
811482,9993764,0,2018-12-30 22:14:00,2018-12-30 23:12:50,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Tow,Black - Prius - 6ZYG936,"2312 35TH AVE, SAN FRANCISCO, CA, 94116",35TH AVE,4.0,Parkside,TARAVAL,37.743806,-122.492778,"(37.74380555, -122.49277774)",Mobile/Open311,1
811494,9993737,0,2018-12-30 21:53:14,2018-12-30 22:18:00,Closed,Case Transferred - See encampment sr#9993757,311 Supervisor Queue,General Request - PUBLIC WORKS,request_for_service,bsm - request_for_service,Intersection of LANDERS ST and 15TH ST,LANDERS ST,8.0,Mission Dolores,MISSION,37.766210,-122.427747,"(37.76620958, -122.42774722)",Mobile/Open311,1


# Latitude and Longitude

In [3]:
# Rounding the lat/long
df['latitude'] = round(df['latitude'], 3)
df['longitude'] = round(df['longitude'], 3)
df.shape

(529820, 20)

## Future Work (cluster coordinates)

In [ ]:
# df['point'] = df['latitude'].apply(str) + ', ' + df['longitude'].apply(str)
# # df['point'] = df['point'].astype('category')
# df.head()
# df.point.nunique()

In [ ]:
# from sklearn.cluster import DBSCAN
# from sklearn import metrics

# test = df.head(10000)

# coords = test.loc[:, ['longitude', 'latitude']].to_numpy()

# # kms_per_radian = 500
# # epsilon = 1.5 / kms_per_radian
# db = DBSCAN(eps=.000019, min_samples=5, algorithm='ball_tree', metric='haversine', n_jobs=-1).fit(np.radians(coords)) # eps=.000019 gives 38 clusters for 1000
# cluster_labels = db.labels_
# num_clusters = len(set(cluster_labels))
# clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

# print('Number of clusters: {}'.format(num_clusters))

In [ ]:
# test = pd.DataFrame(cluster_labels)
# test
# test[0].unique()

# Cyclical Features
We need to represent the cyclical features as (x,y) coordinates on a circle.  
http://blog.davidkaleko.com/feature-engineering-cyclical-features.html

In [4]:
# Year
df['opened_year'] = df['opened'].dt.year.astype('category')

# Month
month = df['opened'].dt.month
df['opened_month_sin'] = np.sin(month*(2.*np.pi/12))
df['opened_month_cos'] = np.cos(month*(2.*np.pi/12))

# Week
week = df['opened'].dt.week
df['opened_week_sin'] = np.sin(week*(2.*np.pi/52))
df['opened_week_cos'] = np.cos(week*(2.*np.pi/52))

# Day of week
day = df['opened'].dt.dayofweek
df['opened_day_sin'] = np.sin(day*(2.*np.pi/7))
df['opened_day_cos'] = np.cos(day*(2.*np.pi/7))

# Hour
hour = df['opened'].dt.hour
df['opened_hour_sin'] = np.sin(hour*(2.*np.pi/24))
df['opened_hour_cos'] = np.cos(hour*(2.*np.pi/24))

In [5]:
df.head()

,case_id,target,opened,closed,status,status_notes,responsible_agency,category,request_type,request_details,...,has_media,opened_year,opened_month_sin,opened_month_cos,opened_week_sin,opened_week_cos,opened_day_sin,opened_day_cos,opened_hour_sin,opened_hour_cos
810969,9995194,1,2018-12-31 10:42:22,2018-12-31 00:00:00,Closed,Case Resolved - duplicate of 21375555 -,DPW - Bureau of Street Use and Mapping - G,General Request - PUBLIC WORKS,request_for_service,bsm - request_for_service,...,0,2018,-2.449294e-16,1.0,1.205367e-01,0.992709,0.000000,1.00000,0.500000,-0.866025
811474,9993791,1,2018-12-30 22:40:00,2018-12-30 23:13:06,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Black - Toyota Camry - NO PLATES,...,1,2018,-2.449294e-16,1.0,-2.449294e-16,1.000000,-0.781831,0.62349,-0.500000,0.866025
811481,9993771,1,2018-12-30 22:18:00,2018-12-30 22:19:29,Closed,Case is Invalid - Contact name and phone numbe...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,silver` - Mazda 3 Hatch back - 7YVN660,...,0,2018,-2.449294e-16,1.0,-2.449294e-16,1.000000,-0.781831,0.62349,-0.500000,0.866025
811482,9993764,0,2018-12-30 22:14:00,2018-12-30 23:12:50,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Tow,Black - Prius - 6ZYG936,...,1,2018,-2.449294e-16,1.0,-2.449294e-16,1.000000,-0.781831,0.62349,-0.500000,0.866025
811494,9993737,0,2018-12-30 21:53:14,2018-12-30 22:18:00,Closed,Case Transferred - See encampment sr#9993757,311 Supervisor Queue,General Request - PUBLIC WORKS,request_for_service,bsm - request_for_service,...,1,2018,-2.449294e-16,1.0,-2.449294e-16,1.000000,-0.781831,0.62349,-0.707107,0.707107


# Resolution Time

In [6]:
# Diff in minutes
df['resolution_time'] = df['closed'] - df['opened']
df['resolution_time'] = (df['resolution_time']/np.timedelta64(1,'m'))
# Remove rows with negative minutes & requests over 7 days
df = df.loc[(df['resolution_time'] > 0)] #& (df['resolution_time'] < 10080)]
df.shape

(529769, 30)

# Media

In [7]:
# # Create 'has_media' column 
# df['has_media'] = df['media_url'].notnull().astype('int8').astype('category')

# Category Groups

In [8]:
re_street = ['street_cleaning', 'sidewalk cleaning']
re_graffiti = ['graffiti', 'graffiti']
re_parking = ['parking', 'abandoned vehicle|parking enforcement|blocked street or sidewalk']
re_general = ['general', 'public works|sewer issues|tree\b|streetlights|curb|noise']
re_home = ['homelessness', 'encampments|homeless']
re_muni = ['muni_feedback', 'muni']

re_list = [re_street, re_graffiti, re_parking, re_general, re_home, re_muni]

for i in re_list:
    df.insert(loc=(len(df.columns)), column=i[0],
              value=df['category'].str.lower().str.contains(i[1], regex=True).astype('int8'))

In [9]:
df.to_pickle('../data/02_df_pre_model_2018.pkl')

# Appendix

## Category Grouping
* General
    * Blocked Pedestrian Walkway
    * Encampment
    * Noise Issue
    * Park Requests
    * Tree Maintenance
* Cleaner Streets
    * Garbage Containers
    * Graffiti
    * Illegal Postings
    * Street or Sidewalk Cleaning
* Parking & Transportation
    * Abandoned Vehicles
    * Blocked Driveway & Illegal Parking
    * Muni Employee Feedback
    * Muni Service Feedback
* Repair
    * Curb & Sidewalk Issues
    * Damaged Public Property
    * Flooding, Sewer & Water Leak Issues
    * Parking & Traffic Sign Repair
    * Pothole & Street Issues
    * Streetlight Repair

In [ ]:
# Distribution of Resolution Time
plt.hist(df['resolution_time'])

In [ ]:
# Top categories by frequency of wasteful requests
df.loc[df['target']==1, 'category'].value_counts()[:10]